In [1]:
######################## 机器学习对数几率回归类 ##############################
# Author: 杨玉奇
# email: yangyuqi@sina.com
# url: https://github.com/jerryyyq/tf_algorithm_example
# copyright yangyuqi
# 著作权归作者 杨玉奇 所有。商业转载请联系作者获得授权，非商业转载请注明出处。
# date: 2017-09-13
###################################################################

from ML_Model import ML_Model
import tensorflow as tf
    
class ML_Sigma_Regression( ML_Model ):
    def __init__(self):
        ML_Model.__init__(self)

        self.__W = tf.Variable(tf.zeros([5, 1]), name = 'weights')
        self.__b = tf.Variable(0., name = 'bias')
    
        
    def combine_inputs(self, features):
        return tf.matmul(features, self.__W) + self.__b
    
    def inference(self, features):
        return tf.sigmoid( self.combine_inputs(features) )
    
    def loss(self, features, label):
        label_predicted = self.combine_inputs(features)
        return tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(label_predicted, label) )
    
    
    def inputs(self, file_name = [], batch_size = 10):
        passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        self._read_csv(batch_size, file_name, [[0.0], [0.0], [0], [''], [''], [0.0], [0.0], [0.0], [''], [0.0], [''], ['']])
        # print('-----inputs: ', passenger_id, survived, pclass, name)

        # return passenger_id, survived
        
        is_first_class = tf.to_float(tf.equal(pclass, [1]))
        is_second_class = tf.to_float(tf.equal(pclass, [2]))
        is_third_class = tf.to_float(tf.equal(pclass, [3]))

        gender = tf.to_float(tf.equal(sex, ['female']))

        features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
        label = tf.reshape(survived, [batch_size, 1])
        return features, label 
        
        
    def train(self, loss):    
        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer( learning_rate ).minimize( loss )

    
    def evaluate(self, test_features, test_label):
        self._echo_tensor(self.__W, 'At evaluate, the __W')
        self._echo_tensor(self.__b, 'At evaluate, the __b')
        self._echo_tensor(test_features, 'At evaluate, test_features')
        self._echo_tensor(test_label, 'At evaluate, test_label')
        
        label_predicted = tf.cast( self.inference(test_features) > 0.5, tf.float32 )
        
        return tf.reduce_mean( tf.cast(tf.equal(label_predicted, test_label), tf.float32) )

    
if __name__ == '__main__':
    one_ml = ML_Sigma_Regression()
    # data from: https://www.kaggle.com/c/titanic/data
    one_ml.do_train( 10, ['Titanic_train.csv'] )

    one_ml.do_evaluate( ['Titanic_train.csv'] )

    

-------------- do_train: start -----------------
file_path =  ['/home/yangyuqi/work/tf_algorithm_example/Titanic_train.csv'] 

Tensor("ReaderRead:0", shape=(), dtype=string) Tensor("ReaderRead:1", shape=(), dtype=string)
features_0 tensor.shape = [10  5], tensor = [[  0.   0.   1.   1.  38.]
 [  0.   0.   1.   0.   0.]
 [  0.   0.   1.   1.  15.]
 [  1.   0.   0.   0.  19.]
 [  0.   0.   1.   1.  14.]
 [  0.   0.   1.   1.   0.]
 [  0.   1.   0.   1.  55.]
 [  0.   0.   1.   1.  27.]
 [  0.   1.   0.   0.  66.]
 [  0.   0.   1.   0.   0.]]

label_0 tensor.shape = [10  1], tensor = [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]

step_0 loss:  tensor.shape = [], tensor = 0.6980220079421997

training end. step_9 final loss:  tensor.shape = [], tensor = 0.8247721791267395

save_path is:  /tmp/ML_Sigma_Regression.vari
At evaluate, the __W tensor.shape = [5 1], tensor = [[ 0.0016798 ]
 [ 0.00054858]
 [-0.00778233]
 [ 0.00935212]
 [-0.12496768]]

At evaluate, the __b t

In [1]:
# 中间计算数据样例展示
import tensorflow as tf

f = [[1., 0., 0., 1., 35.],
 [0., 0., 1., 1., 26.],
 [0., 0., 1., 0., 0.]]

l = [[ 1.], [ 0.], [ 0.]]

tf_f = tf.convert_to_tensor(f)
tf_l = tf.convert_to_tensor(l)


f1 = [1., 0., 0.]
f2 = [0., 0., 0.]
f3 = [0., 1., 1.]


with tf.Session() as sess:
    print( sess.run(tf.shape(f)) )
    # print( sess.run(f) ) --> Can not convert a list into a Tensor or Operation.
    print('f: ', f)
    print( 'tf_f shape: ', sess.run(tf.shape(tf_f)) )
    print( sess.run(tf_f) )
    
    print( 'tf_f,l: ', sess.run([tf_f, tf_l]) )

    f_pack = tf.pack([f1, f2, f3])
    print( 'f_pack: ', f_pack, sess.run(f_pack) )
    f_pack_tr = tf.transpose(f_pack)
    print( 'f_pack_tr: ', sess.run(f_pack_tr) )
    print( 'shape(tf_pack): ', tf.shape(f_pack), sess.run(tf.shape(f_pack)) )

    
    W = tf.Variable(tf.zeros([5, 1]), name = 'weights')
    b = tf.Variable(0., name = 'bias')
    sess.run( tf.initialize_all_variables() )
    
    label_predicted = tf.matmul(f, W) + b   # tf_f, f total OK!
    sig = tf.nn.sigmoid_cross_entropy_with_logits(label_predicted, l)
    loss = tf.reduce_mean( sig ) # tf_l, l total OK!
    print( sess.run([label_predicted, sig, loss]) )
    
    learning_rate = 0.01
    cost = tf.train.GradientDescentOptimizer( learning_rate ).minimize( loss )
    sess.run(cost)
    print(sess.run([W, b]))
    print(sess.run(loss))
    
    sess.close()
    

[3 5]
f:  [[1.0, 0.0, 0.0, 1.0, 35.0], [0.0, 0.0, 1.0, 1.0, 26.0], [0.0, 0.0, 1.0, 0.0, 0.0]]
tf_f shape:  [3 5]
[[  1.   0.   0.   1.  35.]
 [  0.   0.   1.   1.  26.]
 [  0.   0.   1.   0.   0.]]
tf_f,l:  [array([[  1.,   0.,   0.,   1.,  35.],
       [  0.,   0.,   1.,   1.,  26.],
       [  0.,   0.,   1.,   0.,   0.]], dtype=float32), array([[ 1.],
       [ 0.],
       [ 0.]], dtype=float32)]
f_pack:  Tensor("pack:0", shape=(3, 3), dtype=float32) [[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  1.  1.]]
f_pack_tr:  [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
shape(tf_pack):  Tensor("Shape_2:0", shape=(2,), dtype=int32) [3 3]
[array([[ 0.],
       [ 0.],
       [ 0.]], dtype=float32), array([[ 0.69314718],
       [ 0.69314718],
       [ 0.69314718]], dtype=float32), 0.69314718]
[array([[ 0.00166667],
       [ 0.        ],
       [-0.00333333],
       [ 0.        ],
       [ 0.015     ]], dtype=float32), -0.0016666667]
0.686475


In [5]:
tf.shape(tensor)